In [ ]:
# Colab에 필요한 라이브러리를 설치합니다.
# 'accelerate'는 Trainer가 GPU/TPU를 쉽게 사용하도록 도와줍니다.
!pip install transformers datasets accelerate evaluate

In [ ]:
import torch
import numpy as np
from datasets import load_dataset#, load_metric # Removed load_metric from datasets
from evaluate import load as load_metric # Imported load_metric from evaluate

from transformers import (
    BertTokenizer,
    AutoModelForSequenceClassification, # ⭐️ 'AutoModel'이 아닌 'AutoModelForSequenceClassification'을 사용
    Trainer,
    TrainingArguments,
    set_seed
)

# 1. 데이터셋 로드
datasets = load_dataset("dair-ai/emotion")
print(datasets)

# 2. 라벨 정보 확인 (총 6개)
label_names = datasets["train"].features["label"].names
num_labels = len(label_names)
print(f"라벨 수: {num_labels}, 라벨: {label_names}")

# 3. GPU/CPU 설정
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"사용할 장치: {DEVICE}")

# - 실험 재현을 위해 시드를 고정한다
set_seed(42)


In [ ]:
# - 학습 관련 핵심 값을 한 곳에서 정의한다
NUM_TRAIN_EPOCHS = 3
PER_DEVICE_TRAIN_BATCH_SIZE = 32
PER_DEVICE_EVAL_BATCH_SIZE = 32
LEARNING_RATE = 5e-5
LOGGING_STEPS = 100
OUTPUT_DIR = "./results"

# - 현재 설정을 요약해 확인한다
def describe_run():
    print("=== Trainer 설정 ===")
    print(f"epochs: {NUM_TRAIN_EPOCHS}")
    print(f"train_batch_size: {PER_DEVICE_TRAIN_BATCH_SIZE}")
    print(f"eval_batch_size: {PER_DEVICE_EVAL_BATCH_SIZE}")
    print(f"learning_rate: {LEARNING_RATE}")
    print(f"logging_steps: {LOGGING_STEPS}")
    print(f"output_dir: {OUTPUT_DIR}")
    print(f"num_labels: {num_labels}")
    print(f"device: {DEVICE}")

describe_run()


In [ ]:
VOCAB_FILE = "./bert-implementation/data-preprocessing/mini_emotion_tokenizer_7k.txt"
tokenizer = BertTokenizer.from_pretrained(VOCAB_FILE, do_lower_case=True)

def preprocess_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True, max_length=128)

print("커스텀 7k 토크나이저 로드 완료")
tokenized_datasets = datasets.map(preprocess_function, batched=True)
tokenized_datasets.set_format("torch", columns=["input_ids", "attention_mask", "label"])
print("전처리 후 데이터 샘플:")
print(tokenized_datasets["train"][0])


In [ ]:
MODEL_PATH = "./my_bert_hf_model"
model = AutoModelForSequenceClassification.from_pretrained(
    MODEL_PATH,
    num_labels=num_labels
).to(DEVICE)

print("커스텀 Hugging Face 체크포인트 로드 완료.")
print(model.config)


In [ ]:
# 1. 사용할 평가 지표 로드 (여기서는 'accuracy')
metric = load_metric("accuracy")

# 2. 'Trainer'가 사용할 평가 함수 정의
def compute_metrics(eval_pred):
    """
    Trainer가 평가 시 호출하는 함수입니다.
    logits과 labels을 받아 accuracy를 계산합니다.
    """
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)

    return metric.compute(predictions=predictions, references=labels)

print("평가 지표(accuracy) 계산 함수 준비 완료.")

In [ ]:
# - Hugging Face Trainer에 전달할 학습/로깅 옵션을 정의한다
training_args = TrainingArguments(
    output_dir=OUTPUT_DIR,
    # --- 학습 관련 설정 ---
    num_train_epochs=NUM_TRAIN_EPOCHS,
    per_device_train_batch_size=PER_DEVICE_TRAIN_BATCH_SIZE,
    per_device_eval_batch_size=PER_DEVICE_EVAL_BATCH_SIZE,
    learning_rate=LEARNING_RATE,
    # --- 평가 및 저장 설정 ---
    eval_strategy="epoch",
    save_strategy="epoch",
    # --- 기타 설정 ---
    load_best_model_at_end=True,
    metric_for_best_model="accuracy",
    logging_steps=LOGGING_STEPS,
    report_to="none"
)

print(f"'{training_args.output_dir}' 폴더에 결과가 저장됩니다.")


In [ ]:
# 1. Trainer 객체 생성
# 학습에 필요한 모든 재료(모델, 설정, 데이터, 토크나이저, 평가함수)를 넣어줍니다.
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)

# 2. ⭐️ 학습 시작! ⭐️
# 이 한 줄이 Native PyTorch의 긴 '셀 6' 전체를 대체합니다.
print("--- Trainer로 학습 시작 ---")
trainer.train()
print("--- 학습 완료! ---")

In [ ]:
from transformers import pipeline

print("--- 학습된 모델로 Softmax 확률 값 예측 ---")

# 1. 'text-classification' 파이프라인 생성
#    trainer.model은 'load_best_model_at_end=True'에 의해
#    가장 accuracy가 높았던 모델입니다.
classifier_pipeline = pipeline(
    "text-classification",
    model=trainer.model,     # ⭐️ 학습 완료된 베스트 모델
    tokenizer=tokenizer,
    device=0 if torch.cuda.is_available() else -1 # 0: GPU, -1: CPU
)

# 2. 테스트할 문장
test_text_1 = "I feel so happy and excited today!"
test_text_2 = "This is so frustrating and makes me angry."

# 3. 예측 실행 (return_all_scores=True로 모든 라벨 확률 받기)
results_1 = classifier_pipeline(test_text_1, return_all_scores=True)
results_2 = classifier_pipeline(test_text_2, return_all_scores=True)

# 4. 결과 출력
def print_results(text, results):
    print(f"\n입력 문장: \"{text}\"")
    print("--- 6개 라벨 Softmax 확률 값 ---")

    # 라벨 이름을 매칭시켜서 보기 좋게 출력
    for res in results[0]:
        label_name = label_names[int(res['label'].split('_')[-1])]
        print(f"{label_name:10}: {res['score']:.4f} ( {res['score']*100:6.2f} % )")

print_results(test_text_1, results_1)
print_results(test_text_2, results_2)